# Dependences

In [1]:
include("../../src/struct_data.jl")

save_data

# Model

In [2]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

dump(model)

  0.000015 seconds (5 allocations: 208 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 150000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_1"
    path_output: String ""


# Aggregate

In [3]:
agg = nothing
# @time @start_agg FusionAGG = FusionAggregate(
@time agg = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        ),
        AggType(
            "HEK_2", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    model
)
# show_aggregates(agg)

  7.245348 seconds (7.60 M allocations: 606.663 MiB, 4.64% gc time)


Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}), AggType("HEK_2", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2"]), Float32[-15.77 -4.62 -13.88; -13.77 -4.62 -13.88; … ; 14.77 4.62 13.88; 16.77 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.

# kNN

In [4]:
include("../../src/neighbor.jl")
println("------------------------ SIZE ------------------------")
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    shmem=prod(threads.+2)*sizeof(Float32),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   

println("------------------------ IDX -------------------------")
display(agg.Simulation.Neighbor.idx_red)

------------------------ SIZE ------------------------
------------------------ IDX -------------------------


1000×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 36  37  38  41  42  43  44  45  57  …  4965  4966  4967  4972  4973  4974
  6   7   8  49   1   2   3   3  58     5006  5007  5008  5002  5003  5004
  5   6   7  50  50   1   2  53  15     5007  5008  4959  5003  5004  5005
 42  43  44   5  51  51  52  54  10     4958  4959  4960  4965  4966  4967
 43  44  45  10   6  52  53   7  49     4957  4958  5004  4964  4965  4966
  2   3   2   9   4   7   8  14   4  …  5002  5003  4999  5007  5006  5007
 35   1  37  40  11   5   6  13  66     5004  5005  5000  4973  5008  4975
 37  36  39  42  10  12  13  44  50     4997  4998  4966  4971  4972  4973
 51  38  53  58  41  11  12  46  48     4998  4999  4968  4957  4974  4959
  0  52   0   0  43  42  43  62   0     4964  4965  4951     0  4958     0
  0   0   0   0  59  44  45   0   0  …  4966  4967     0     0     0     0
  0   0   0   0   0  60  61   0   0     4949  4950     0     0     0     0
  0   0   0   0   0   0   0   0   0        0    

# Force

## Function

In [109]:
include("../../src/forces/forces.jl")

threads=(64,3)
CUDA.@time @cuda(
    threads = threads,
    blocks = cld.(size(agg.Position,),threads),
    shmem=prod(threads.+2)*sizeof(Float32),
    sum_force!(
        agg.Position,
        agg.Simulation.Force.F,
        agg.Simulation.Force.Pol,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Parameter.Force,
        agg.Simulation.Parameter.Contractile.fₚ,
        atan(1),
        cos(pi/3),
        model.Time.dt
    )
)

display(agg.Simulation.Force.F)
display(agg.Position)

  0.664232 seconds (425.61 k CPU allocations: 21.675 MiB)


5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.000343683  -0.000804524   0.00217834
 -0.00192004    9.49909f-5    0.000496646
  0.000857607  -0.000227333   0.000486911
  7.94849f-5    9.09941f-5    0.000339867
  0.000760225  -0.000894519   0.00073076
  1.70067f-5    5.10882f-6    0.00027893
  0.000131841  -0.00109986   -0.00024003
 -0.000173181   0.00122763    0.000779471
 -0.000792493  -0.0009112     0.00123763
 -0.0010657     0.0012037     0.00200908
  0.0003555    -0.000951028  -0.000349401
 -0.000786957   0.000386862   0.00268051
 -0.000909325  -0.000665849  -0.00194618
  ⋮                          
 -0.00205092    0.000663393  -0.000869943
  0.00095276   -4.38057f-5   -0.000220083
  0.000162065   9.10758f-5   -0.00028696
 -0.000114606   1.74376f-5   -0.000260867
  0.000515796   0.00106938   -0.00152418
 -0.000117118  -0.000144619  -0.000328997
  2.54273f-5    2.26527f-5   -0.000196898
 -0.000135551   0.0012455    -0.00303445
 -0.000697599   0.000796961  -0.0015551
 -0.0015

5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -15.7754   -4.62317  -13.8639
 -13.7719   -4.62016  -13.8673
 -11.7638   -4.62089  -13.8644
 -18.7761   -2.8871   -13.8673
 -16.7688   -2.8886   -13.8659
 -14.77     -2.88894  -13.871
 -12.7683   -2.88944  -13.8709
 -10.7667   -2.88684  -13.867
 -19.7747   -1.1517   -13.8677
 -17.7693   -1.15297  -13.8675
 -15.7676   -1.15395  -13.8689
 -13.7726   -1.15325  -13.866
 -11.7698   -1.15183  -13.8722
   ⋮                  
  12.7705    1.15619   13.8681
  14.7707    1.14985   13.8678
  16.7683    1.15397   13.8701
  18.7742    1.15392   13.8652
   9.76678   2.8885    13.8657
  11.77      2.88956   13.8703
  13.7693    2.88693   13.866
  15.7701    2.88633   13.8684
  17.7722    2.89128   13.8688
  12.7644    4.62204   13.8643
  14.7701    4.62088   13.8687
  16.7736    4.61851   13.8695